In [1]:
import random
import numpy as np

def ransac_line_fitting(X, k, t, d, e):
    """
    RANSAC 알고리즘으로 직선을 추정하는 함수
    :param X: 데이터 점 집합 [(a1, b1), (a2, b2), ..., (an, bn)]
    :param k: 반복 횟수
    :param t: 인라이어 판별 임계값 (거리)
    :param d: 최소 인라이어 수
    :param e: 직선 오차
    :return: 최적의 직선 모델 T (기울기 m, 절편 c)
    """
    Q = []  # 최종 인라이어 집합
    best_T = None  # 최적의 직선 모델 (m, c)
    max_inliers = 0  # 최대 인라이어 수

    for _ in range(k):
        # 1. 두 점을 무작위로 선택
        idx1, idx2 = random.sample(range(len(X)), 2)
        p1, p2 = X[idx1], X[idx2]

        # 2. 두 점으로 직선 방정식 계산 (y = mx + c)
        # 기울기 m = (y2 - y1) / (x2 - x1)
        if p2[0] - p1[0] == 0:  # x값이 같으면 수직선이므로 스킵
            continue
        m = (p2[1] - p1[1]) / (p2[0] - p1[0])
        # 절편 c = y - mx
        c = p1[1] - m * p1[0]
        T = (m, c)  # 직선 모델 T: (기울기, 절편)

        # 3. 모든 점에 대해 직선과의 거리 계산 및 인라이어 집합 구성
        inliers = []
        for p in X:
            x, y = p
            # 점 (x, y)와 직선 y = mx + c 사이의 거리
            # 거리 = |y - (mx + c)| / sqrt(m^2 + 1)
            distance = abs(y - (m * x + c)) / np.sqrt(m**2 + 1)
            if distance < t:  # 거리가 임계값 t보다 작으면 인라이어
                inliers.append(p)

        # 4. 인라이어 수가 d 이상인지 확인
        if len(inliers) >= d:
            # 5. 인라이어 집합으로 직선을 다시 피팅 (최소제곱법 사용)
            if len(inliers) >= 2:
                # 인라이어 점들로 최소제곱법 적용
                x_vals = [p[0] for p in inliers]
                y_vals = [p[1] for p in inliers]
                A = np.vstack([x_vals, np.ones(len(x_vals))]).T
                m_new, c_new = np.linalg.lstsq(A, y_vals, rcond=None)[0]
                T = (m_new, c_new)

                # 6. 새로운 직선으로 인라이어 다시 계산
                inliers = []
                for p in X:
                    x, y = p
                    distance = abs(y - (m_new * x + c_new)) / np.sqrt(m_new**2 + 1)
                    if distance < t:
                        inliers.append(p)

            # 7. 인라이어 수가 충분히 크면 최적 모델 갱신
            if len(inliers) > max_inliers:
                max_inliers = len(inliers)
                best_T = T
                Q = inliers

    # 8. 최종 직선 모델 반환
    if best_T is None or len(Q) < e:
        return None, Q  # 직선 오차 e보다 인라이어가 적으면 None 반환
    return best_T, Q

# 테스트
X = [(1, 2), (2, 4), (3, 6), (4, 8), (5, 10), (2, 1), (3, 2)]  # 데이터 점 (일부 이상치 포함)
k = 100  # 반복 횟수
t = 1.0  # 인라이어 판별 임계값
d = 3  # 최소 인라이어 수
e = 2  # 직선 오차 (최소 인라이어 수)

best_T, inliers = ransac_line_fitting(X, k, t, d, e)
if best_T:
    m, c = best_T
    print(f"최적 직선: y = {m:.2f}x + {c:.2f}")
    print(f"인라이어 점: {inliers}")
else:
    print("적절한 직선을 찾지 못했습니다.")

최적 직선: y = 2.61x + -3.10
인라이어 점: [(1, 2), (2, 4), (3, 6), (4, 8), (5, 10), (2, 1), (3, 2)]
